In [1]:
import pandas as pd

In [2]:
filepath = '/Volumes/backup_128G/z_repository/TBIO_data/RequestsFromTana/all_org_classes'

v10_tana = 'output_class_to_orgs_20190501_v10_tana.xlsx'

read_v10_tana = '{0}/{1}'.format(filepath, v10_tana)

In [3]:
workDf = pd.read_excel(read_v10_tana)
workDf = workDf.fillna(0)
workDf.shape, workDf.head()

((19439, 5),
      Graph   Organization        SocietalSector                        Period  \
 0  chrj_pr  _台灣青年_刊物任發行編輯             MassMedia      EarlyPostWarOrganization   
 1  chrj_pr         艋舺保良分局  PublicAdministration     UnknownPeriodOrganization   
 2  chrj_pr         鹿港保良總局  PublicAdministration     UnknownPeriodOrganization   
 3  chrj_pr         林本源製糖社       PrivateBusiness  JapaneseColonialOrganization   
 4  chrj_pr            阿緱廳  PublicAdministration  JapaneseColonialOrganization   
 
   Correct name  
 0      台灣青年雜誌社  
 1            0  
 2            0  
 3            0  
 4            0  )

In [4]:
chrjprList = []

for idx in range(0, len(workDf)):
    row = workDf.loc[idx]

    graph = str(row['Graph'])
    if graph != 'chrj_pr':
        continue

#     oriOrg = str(row['Organization'])
#     societal = str(row['SocietalSector'])
#     period = str(row['Period'])
#     newOrg = str(row['Correct name'])

    chrjprList.append(row)


In [5]:
# import requests
    
# endpoint_url = "http://localhost:4001/u"

#     data = {'db':'tbio', 'query':query, 'limit':100, 'offset':0, 'reasoning':False}

#     r = requests.post(endpoint_url, data=data)
#     print(oriOrg, newOrg, r.status_code, r.reason)

In [1]:
import stardog
import json

adminFile = '/Users/vincent/Projects/TBIO/tbio-conn-admin.json'
conn_details = {}
with open(adminFile, 'r') as readFile:
    conn_details = json.loads(readFile.read())


In [1]:
import json

def sparqlQry(oriOrg, newOrg, societal, period):
    return """DELETE {
    GRAPH ?g {
        ?orgEvt ?evtOp ?org .
    }
    GRAPH ?g2 {
        ?org ?orgOp ?orgO .
    }
}
INSERT {
    GRAPH ?g {
        <http://tbio.orient.cas.cz#%s> a tbio:%s, tbio:%s ;
            <http://www.w3.org/2000/01/rdf-schema#label> "%s"@zh .
        ?orgEvt ?evtOp <http://tbio.orient.cas.cz#%s> .
    }
}
WHERE {
    GRAPH ?g {
        ?person a tbio:Person .
        ?person ?op ?orgEvt .
        ?orgEvt ?evtOp ?org .
        FILTER(sameterm(?org, <http://tbio.orient.cas.cz#%s>)) .
        GRAPH ?g2 {
            ?org ?orgOp ?orgO .
        }
    }
}""" % (newOrg, societal, period, newOrg, newOrg, oriOrg)

# with stardog.Admin(**conn_details) as admin:
#   db = admin.new_database('db')

with stardog.Connection('tbio', **conn_details) as conn:
#     conn.begin()

    idx = 0
    for chrjpr in chrjprList:
        oriOrg = str(chrjpr['Organization'])
        societal = str(chrjpr['SocietalSector'])
        period = str(chrjpr['Period'])
        newOrg = str(chrjpr['Correct name'])

        if newOrg == '0':
            query = sparqlQry(oriOrg, oriOrg, societal, period)
        else:
            query = sparqlQry(oriOrg, newOrg, societal, period)

        results = conn.update(query)
        jsonRes = json.loads(results)
        print(idx, oriOrg, newOrg, jsonRes)
#         print(results)
        idx += 1

# returned_value = os.system(cmd)  # returns the exit code in unix
# print('returned value:', returned_value)

NameError: name 'stardog' is not defined

# remove deleted list

In [8]:
outfile = 'orgs_deleted_20190508_v1.xlsx'
delfile = '{0}/{1}'.format(filepath, outfile)
delDf = pd.read_excel(delfile)
delDf = delDf.fillna(0)
delDf.shape, delDf.head()

((2269, 1),                                 deleted
 0                                     0
 1  (公司設在竹東郡蕃地油羅山由台北海山郡及文山部的三井製材所提供原料),製
 2                                     )
 3                            +_後在嘉義設立分店
 4                          +_獲特選第一名及台展賞)

In [9]:
def delSparqlQryS(org):
    return """DELETE {
    GRAPH ?g {
        ?s ?p ?o .
    }
}
WHERE {
    GRAPH ?g {
        ?s ?p ?o .
        FILTER(sameterm(?s, <http://tbio.orient.cas.cz#%s>)) .
    }
}""" % (org)

def delSparqlQryO(org):
    return """DELETE {
    GRAPH ?g {
        ?s ?p ?o .
    }
}
WHERE {
    GRAPH ?g {
        ?s ?p ?o .
        FILTER(sameterm(?o, <http://tbio.orient.cas.cz#%s>)) .
    }
}""" % (org)

In [10]:
# for idx in range(0, len(delDf)):
#     row = delDf.loc[idx]

#     org = str(row["deleted"])
#     print(org)

In [11]:
with stardog.Connection('tbio', **conn_details) as conn:
#     conn.begin()
    for idx in range(0, len(delDf)):
        row = delDf.loc[idx]

        org = str(row["deleted"])
        
        if org == '0':
            continue
        else:
            query = delSparqlQryS(org)
            results = conn.update(query)
            
            query = delSparqlQryO(org)
            results = conn.update(query)
            
        print(idx, org, results)


1 (公司設在竹東郡蕃地油羅山由台北海山郡及文山部的三井製材所提供原料),製 None
2 ) None
3 +_後在嘉義設立分店 None
4 +_獲特選第一名及台展賞 None
5 ,九曲堂,林子邊,旗後,楠梓,大樹等位 None
6 ,母校 None
7 ,鐵道部竹南門改良線及基隆八堵間縱貫道路,埔里道路,澤井等) None
8 .01_陞該社營業 None
9 .04_部長從事赤糖製造業 None
10 .07_新竹州方面委員會 None
11 .07新竹州方面委員會 None
12 .10_陞社 None
13 12月任台灣警總司令部戰俘管理處處 None
14 13甲 None
15 14歲 None
16 15,16歲林家一房,二房,三房會計長 None
17 15歲時在大甲辨務署 None
18 15歲起在台東經營雜貨商 None
19 16歲入實業界 None
20 16歲時轉苗栗辨務署 None
21 16歲繼承父業 None
22 16歲金合鐵工廠(機械類,鑄物研究) None
23 16歲金合鐵工廠機械類,鑄物研究 None
24 17,18歲來台,研究茶業 None
25 17歲起經營新合吉醬油園,從事醬油製造販賣,為新合吉醬油園 None
26 18歲起獨立經營綢緞商,兼營木材商 None
27 18歲鐵道部 None
28 192?_經營公共汽車業,七輛汽車行駛於北斗,沙山之間 None
29 192?經營公共汽車業,七輛汽車行駛於北斗,沙山之間 None
30 19歲入社會在包工業,周受祿手下八年,李旁生手下五年 None
31 1_日軍領台時,率先到基隆迎接日軍,艋舺保良分局長,鹿港保良總局長,奉樺山總督 None
32 1日軍領台時,率先到基隆迎接日軍,艋舺保良分局長,鹿港保良總局長,奉樺山總督 None
33 20-25歲從事土木包工業 None
34 20歲來台,居於鳳山,開業醫,兼賣藥材 None
35 20歲來台,經營雜貨店 None
36 20歲協德商店見習 None
37 20歲時與兄弟三人共同經營油紙,綢緞商等 None
38 20歲林本源製糖社長,經營金融業土地買賣事業 None
39 20歲經營飲食店 None
40 20歲起 None
41 20餘歲時來台 None
42 21歲時創立豐成商行,為豐成商行老闆 

388 基督教徒 None
389 基隆分會 None
390 基隆市稅 None
391 報社 None
392 增資八百萬圓,擴張銀行業務 None
393 外交家 None
394 外國輸出入商 None
395 大同 None
396 大商行老闆 None
397 大園庄青年 None
398 大園庄養豬改良組合長,大園蘆竹漁業組合長,大園庄業佃 None
399 大成火災 None
400 大成火災,海上,台灣土地建築各 None
401 大永及日星兩會社的 None
402 大甲帽蓆同業組合評議員,兼大甲信用組合 None
403 大甲街常設委員,株式會社永豐商店 None
404 大肚上堡大總理,兼保良局 None
405 大肚上堡大總理兼保良局 None
406 大舞台株式會社監查,台南協會理事,東京電氣器具製造株式會社董事,台南公會 None
407 大裕茶行,集大成木材商行改組為株式會社任執行 None
408 大阪商船南中國航路廈門丸買辦,二年 None
409 大隘水利 None
410 太平 None
411 如水社委員 None
412 委員部分區委員 None
413 婦產科名醫 None
414 學 None
415 學務委員 None
416 學校 None
417 官選州議員 None
418 官選州議員,台中第二區州議員 None
419 宮前町區長,暨台灣農林株式會社執行董事,台北市 None
420 實業家 None
421 對中日親善有貢獻 None
422 屏東市社會教化委員 None
423 山地人教化 None
424 岡山開業醫 None
425 州 None
426 州協議會 None
427 州國防義會評議會 None
428 州市協議會 None
429 州所得稅調查委員會 None
430 州支部參與 None
431 州教化聯合會評議員 None
432 州社會教育委員會 None
433 州農會肥料保管指定所 None
434 工業研究所 None
435 巴黎秋天的畫展入選,台灣人都以他為榮,此時他僅19或20歲 None
436 市 None
437 市勸業協會 None
438 市協議會 None
439 市參議會 None
440 市學 None
441 市所得稅調查委員會 None
442 市

816 經營土地買賣業 None
817 經營土地買賣致富 None
818 經營土地開墾事業 None
819 經營土地開墾事業(西螺) None
820 經營土地開墾事業西螺 None
821 經營土城大安寮的農園 None
822 經營土木包工業,兼從事土地開墾 None
823 經營外眉庄眉山農產,栽培南洋種鳳梨,鳳梨罐頭銷遍全島 None
824 經營多家公司 None
825 經營大分水泥台灣中南部一手販賣店,大分水泥台灣中南部販賣商 None
826 經營大甲帽子輸出販賣業 None
827 經營大甲至后里之間的公共汽車 None
828 經營大稻埕鳳梨罐頭 None
829 經營太平內科,小兒科,花柳病科醫院 None
830 經營家具工業 None
831 經營專賣局煙草用品 None
832 經營對大陸貿易業 None
833 經營小店鋪 None
834 經營山林開墾業 None
835 經營山產等物販賣,兼經營鴉片,煙膏零售商 None
836 經營工廠 None
837 經營布,酒專賣業,後經營釀酒工廠 None
838 經營帽子原料工廠 None
839 經營帽子業 None
840 經營帽蓆商 None
841 經營帽蓆商,木屐的製造販賣並外銷日本 None
842 經營帽蓆業從事林投帽貿易,以輸出大阪,神戶為主 None
843 經營帽蓆輸出商 None
844 經營店鋪 None
845 經營房屋 None
846 經營房屋及土地 None
847 經營拓殖業 None
848 經營文具批發商 None
849 經營文具用品商 None
850 經營日月潭線的公共汽車 None
851 經營日本石油特約店,馬達代理店及砂糖,肥料商等 None
852 經營日用品食品雜貨商 None
853 經營日用雜貨 None
854 經營日用雜貨品販賣及碾米米穀商 None
855 經營日用雜貨商 None
856 經營日用雜貨商及土地 None
857 經營會社 None
858 經營木材 None
859 經營木材及水泥,與友人共組勝和公司,任代表者,經營米業 None
860 經營木材業及石炭業(慈善事業熱心,淡水公會堂建設捐一萬圓) None
861 經營木村組包工業 None
862 經營木炭中盤商特約商,保正 None
863 經營

1204 繼許丙之後,任庶務 None
1205 繼陳定國之後,任汐止街 None
1206 續連任庄協議會 None
1207 羅東刻印同業 None
1208 羅東專賣品零售 None
1209 羅東街市場及屠獸場指定管理人,水租管理委員會 None
1210 羅東街水道係 None
1211 羅東酒零售 None
1212 羅漢外門里 None
1213 羊豬中間商 None
1214 美國,瑞士等國 None
1215 美國芝加哥盧易氏鐵路 None
1216 美術家 None
1217 義倉 None
1218 義勇 None
1219 義民 None
1220 義竹 None
1221 習俗改良會 None
1222 習會 None
1223 老東勢共同苗代 None
1224 老梅分校醫務 None
1225 耳鼻咽喉科研究 None
1226 聯合會下潭支 None
1227 聯合會長 None
1228 聯合業佃會調停委員會 None
1229 聯合社 None
1230 聯盟 None
1231 聯盟地方委員會 None
1232 聯盟會 None
1233 職 None
1234 肥料及鴉片販賣 None
1235 肥料商 None
1236 肥皂) None
1237 肥麵粉公會委員會 None
1238 胡厝寮區委員 None
1239 能鑑別銅瓷骨董 None
1240 臨時勸業委員 None
1241 臨時國勢調查員 None
1242 臨時土地整理委員會 None
1243 臨時害蟲巡視員 None
1244 臨時戶口調查事務 None
1245 臨時戶口調查委員 None
1246 臨時戶口調查委員,兼 None
1247 臨時戶口調查委員會 None
1248 臨時戶口調查委會 None
1249 臨時戶口調查附 None
1250 臨時總會 None
1251 自創米穀公司 None
1252 自助會 None
1253 自勵會 None
1254 自宅從 None
1255 自家 None
1256 自家從事鴉片,煙膏,煙草零售商 None
1257 自家從事鴉片煙膏零售商 None
1258 自家經營官鹽中盤商 None
1259 自家經營日用雜貨商 None
1260 自家經營陶器類貿易商(主要貿易對象是日本及中國) No

1620 購料委員會 None
1621 購砂礫搜集機械船從事新店溪砂利棌集 None
1622 購買 None
1623 購買販賣利用 None
1624 赤島社社員 None
1625 赴上海 None
1626 赴上海,參與當地政治與經濟 None
1627 赴上海,滿州各地工業視察 None
1628 赴上海唸書 None
1629 赴上海視察 None
1630 赴中國任教員 None
1631 赴中國大陸 None
1632 赴中國大陸,上海復旦大學生物學教授兼生物教授 None
1633 赴中國大陸旅行 None
1634 赴中國汕頭做慈善事業的視察 None
1635 赴中國湖南開發爆竹的市場 None
1636 赴九州採金 None
1637 赴南中國及南洋旅行視察 None
1638 赴南京任國民政府北京某要職 None
1639 赴南洋採錫,經營礦業,因病回台後在台中廳警察界服務 None
1640 赴南洋新金山視察 None
1641 赴南洋經商 None
1642 赴南洋諸島視察 None
1643 赴台北參列於揚文會議並赴東京參加慶典 None
1644 赴大陸福建省在廈門農林試驗場任技術師 None
1645 赴大陸經營礦山事業(十年) None
1646 赴天津,中國事變後加入北京新政府,任實業部商工科 None
1647 赴寧波設立餅店 None
1648 赴屏東,從事台灣製糖會社苦力供給包工業 None
1649 赴山梨縣研究加州葡萄 None
1650 赴廈門 None
1651 赴廈門,轉南京,上海 None
1652 赴廈門,陳逸松律師駐廈門事務 None
1653 赴廈門投身實業界 None
1654 赴廈門經營齒科材料商(後大阪,上海設立分店) None
1655 赴新加坡及南洋各地實地診療 None
1656 赴日 None
1657 赴日,醫院暫時休業 None
1658 赴日本,中國,韓國各地遊覽視察 None
1659 赴日本,朝鮮,中國各地遊歷數年 None
1660 赴日本做社會事業視察 None
1661 赴日本優良町村視察 None
1662 赴日本優良農村視察 None
1663 赴日本及中國各地醫術研究 None
1664 赴日本及中國視察商業 None
1665 赴日本各地視察 None
1666 

2036 阿緱廳枋寮 None
2037 阿緱廳民事調停 None
2038 阿緱產業會社及阿緱拓殖會社的高級職員 None
2039 阿緱街開業醫 None
2040 陳悅記祭祀公業管理人 None
2041 陸軍 None
2042 隘藔溪昌基堤坊築造 None
2043 隨內藤大佐遊學日本 None
2044 隨本省工礦處 None
2045 隨林松壽遊學東京 None
2046 隨水野長官赴日本關光 None
2047 隨父母遷居東京 None
2048 集集信購利 None
2049 集集街社子開業醫 None
2050 集鹿拓殖株式會 None
2051 雕刻家 None
2052 雙溪庄 None
2053 雙溪庄義修礦會計,火藥科主任,同坑內計劃 None
2054 雙連潭 None
2055 雜穀商 None
2056 雜貨批發 None
2057 雜貨批發商 None
2058 雜貨經營商 None
2059 雲林,林杞埔,斗六,北斗等地匪徒猖獗叛亂去鎮撫 None
2060 雲泉炭礦 None
2061 電視臺 None
2062 霧峰 None
2063 霧峰一新會委員 None
2064 霧峰商業奉 None
2065 霧峰庄分會 None
2066 霧峰庄方面委員 None
2067 霧峰庄林家的一族 None
2068 霧峰庄財務 None
2069 霧峰開業醫 None
2070 青 None
2071 青年 None
2072 青年團 None
2073 青年團長,州稅調查委員會 None
2074 青果同業 None
2075 青果界 None
2076 青果界,芭蕉的中間商 None
2077 青果聯合代表員 None
2078 青果輸出業 None
2079 青雲博愛會理事 None
2080 靜岡縣地方事務官 None
2081 音樂,汽車,機械類研究 None
2082 音樂家 None
2083 頂六第二 None
2084 頂店庄保 None
2085 頂橋仔頭庄及下橋仔頭庄的總理授其維持治安之責 None
2086 順安屠場事務 None
2087 領台當時軍方司令部有功,漸次發跡 None
2088 領台草草憲兵屯所自衛團組織委員 None
2089 頭分區區 None
2090 頭分街街協 None
2091 頭